# Part 1

Walking along the memory banks of the stream, you find a small village that is experiencing a little confusion: some programs can't communicate with each other.

Programs in this village communicate using a fixed system of pipes. Messages are passed between programs using these pipes, but most programs aren't connected to each other directly. Instead, programs pass messages between each other until the message reaches the intended recipient.

For some reason, though, some of these messages aren't ever reaching their intended recipient, and the programs suspect that some pipes are missing. They would like you to investigate.

You walk through the village and record the ID of each program and the IDs with which it can communicate directly (your puzzle input). Each program has one or more programs with which it can communicate, and these pipes are bidirectional; if 8 says it can communicate with 11, then 11 will say it can communicate with 8.

You need to figure out how many programs are in the group that contains program ID 0.

For example, suppose you go door-to-door like a travelling salesman and record the following list:

```
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
```

In this example, the following programs are in the group that contains program ID 0:

Program 0 by definition.
Program 2, directly connected to program 0.
Program 3 via program 2.
Program 4 via program 2.
Program 5 via programs 6, then 4, then 2.
Program 6 via programs 4, then 2.
Therefore, a total of 6 programs are in this group; all but program 1, which has a pipe that connects it to itself.

How many programs are in the group that contains program ID 0?

In [16]:
import textwrap
from io import StringIO

import networkx

In [2]:
def programs_and_pipes(lines):
    for line in lines:
        program, pipes = line.split(' <-> ')
        program = int(program)
        pipes = {int(x) for x in pipes.split(', ')}
        yield program, pipes

In [13]:
def test_input():
    yield from programs_and_pipes(StringIO(textwrap.dedent(
        """\
        0 <-> 2
        1 <-> 1
        2 <-> 0, 3, 4
        3 <-> 2, 4
        4 <-> 2, 3, 6
        5 <-> 6
        6 <-> 4, 5
        """)))

In [15]:
def puzzle_input():
    with open('./inputs/day12/input.txt') as f:
        yield from programs_and_pipes(f)

## networkx solution

In [26]:
def build_graph(pairs) -> networkx.Graph:
    graph = networkx.Graph()
    graph.add_edges_from((program, pipe) for program, pipes in pairs for pipe in pipes)
    return graph

In [27]:
assert len(networkx.node_connected_component(build_graph(test_input()), 0)) == 6

In [28]:
len(networkx.node_connected_component(build_graph(puzzle_input()), 0))

378

## Plain-Python solution

In [56]:
def build_network(graph_dict: dict, node: int, network: set=None) -> int:
    network = network or set()
    if node in network:
        return
    network.add(node)
    for connected_node in graph_dict[node]:
        build_network(graph_dict, connected_node, network)
    
    return network

In [57]:
assert len(build_network({program: pipes for program, pipes in test_input()}, 0)) == 6

In [58]:
len(build_network({program: pipes for program, pipes in puzzle_input()}, 0))

378

# Part 2

There are more programs than just the ones in the group containing program ID 0. The rest of them have no way of reaching that group, and still might have no way of reaching each other.

A group is a collection of programs that can all communicate via pipes either directly or indirectly. The programs you identified just a moment ago are all part of the same group. Now, they would like you to determine the total number of groups.

In the example above, there were 2 groups: one consisting of programs 0,2,3,4,5,6, and the other consisting solely of program 1.

How many groups are there in total?

## networkx solution

In [30]:
assert networkx.number_connected_components(build_graph(test_input())) == 2

In [32]:
networkx.number_connected_components(build_graph(puzzle_input()))

204

## Plain-Python solution

In [67]:
def subnetworks(graph_dict: dict):
    nodes = set(graph_dict.keys())
    node = 0
    subgraphs = []
    while True:
        subgraphs.append(build_network(graph_dict, node))
        nodes -= subgraphs[-1]
        if nodes:
            node = nodes.pop()
        else:
            break
    return subgraphs

In [68]:
assert len(subnetworks({program: pipes for program, pipes in test_input()})) == 2

In [69]:
len(subnetworks({program: pipes for program, pipes in puzzle_input()}))

204